# Day 7

Another conference day, so explanations will come later.

In [81]:
from utils import read_input
from collections import namedtuple

Command = namedtuple('Command', ['command', 'target'], defaults=(None, None))
Directory = namedtuple('Directory', ['name'])
File = namedtuple('File', ['name', 'size'])

def transformer(line):
    if line.startswith('$'):
        prompt = line.split(' ')
        if len(prompt) == 2:
            return Command(command=prompt[1])
        elif len(prompt) == 3:
            return Command(command=prompt[1], target=prompt[2])
    elif line[0].isnumeric():
        size, name = line.split(' ')
        return File(name, int(size))
    elif line.startswith('dir'):
        _, name = line.split(' ')
        return Directory(name)
    else:
        raise Exception('Unknown line')
        
listings = read_input(7, transformer)
examples = read_input(7, transformer, True)

class FileObj:
    
    def __init__(self, name, size, is_dir):
        self.name = name
        self.is_dir = is_dir
        self.size = size
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        
    def calculate_size(self):
        s = self.size
        for child in self.children:
            s += child.calculate_size()
        return s
    
    def __repr__(self):
        if self.is_dir:
            return f'<Dir name="{f"/{self.name}"}" children={self.children} size={self.calculate_size()}>'
        else:
            return f'<File name="{self.name}" size={self.size}>'

def process(listings):
    filesystem = {'/': FileObj('/', 0, True)}
    pwd = []
    for listing in listings:
        obj = None
        match listing:
            case Directory(name):
                obj = FileObj(name, 0, True)
                path = '/'.join(pwd + [name]).replace('//', '/')
                filesystem[path] = obj
                parent = filesystem['/'.join(pwd).replace('//', '/')]
                parent.add_child(obj)
            case File(name=name, size=size):
                obj = FileObj(name, size, False)
                path = '/'.join(pwd).replace('//', '/')
                parent = filesystem[path]
                parent.add_child(obj)
            case Command('ls', None):
                continue
            case Command('cd', '..'):
                pwd.pop()
            case Command('cd', None):
                pwd = ['/']
            case Command('cd', directory):
                pwd.append(directory)
    return filesystem

In [80]:
MAX_SIZE = 100000
filesystem = process(listings.copy())

dirs = [f for f in filesystem.values() if f.is_dir]
small_dirs = [d for d in dirs if d.calculate_size() <= MAX_SIZE]

solution_1 = sum(d.calculate_size() for d in small_dirs)
print(f'Part 1: {solution_1}')
assert solution_1 == 1350966

Part 1: 1350966


## Part 2

In [95]:
TOTAL_DISK_SPACE = 70000000
AVAILABLE_NEEDED = 30000000

filesystem = process(listings.copy())

current_space = filesystem['/'].calculate_size()
space_needed = AVAILABLE_NEEDED - (TOTAL_DISK_SPACE - current_space)

solution_2 = min(
    (d.calculate_size() 
     for d 
     in filesystem.values() 
     if d.is_dir and d.calculate_size() >= space_needed
    )
)

print(f'Part 2: {solution_2}')
assert solution_2 == 6296435

Part 2: 6296435
